In [1]:
import pandas as pd
import numpy as np
from pandas import *
from scipy import stats
import os
import re
import math

In [2]:
# Warning: this will probably take some time
# Adjust file path based on your computer

file = '../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Help-Seeking/Data/p8_data_new.csv'
data = pd.read_csv(file)

/anaconda3/envs/caispp/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (2,4,6,21,22,23,24,25) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
### DATA PREPROCESSING

# 1. one-hot encode categorical data
# 2. standardize variables 
# 3. fill in null values

In [4]:
# What are non open face/open pose columns? 
# Feature Dictionary Documentation here: https://docs.google.com/document/d/1RSygoLwsM1PKEIOoqDOaEo3lRBZBLZJuzbMSY5K_6zA/edit?usp=sharing

for i in data.columns:
    # open face/open pose columns begin with of_ or op_
    if 'op' not in i and 'of' not in i:
        print(i)

participant
session_num
session_date
timestamp
engagement
activity
skill
difficulty
aptitude
games_total
games_session
mistakes_total
mistakes_session
mistakes_game
ts_robot_talked
ts_game_start
ts_attempt
game_start
game_correct
game_incorrect
mistake_made
ros_PARTICIPANT_STATE
ros_ROBOT_STATE
transcript_spk_0
transcript_spk_1
transcript_spk_2


In [5]:
# Defintely drop the following columns 

# transcriptions
#data = data.drop(columns=['transcript_spk_0', 'transcript_spk_1', 'transcript_spk_2'])

# raw ros messages
#data = data.drop(columns=['ros_PARTICIPANT_STATE', 'ros_ROBOT_STATE'])

# more for counting purposes
#data = data.drop(columns=['game_start', 'game_correct', 'game_incorrect', 'mistake_made'])

#data = data.drop(columns=['participant', 'session_date'])

# difficulty, skill, activity

In [6]:
# One Hot Encoding for Skill and Difficulty

data['skill_NC'] = 0
data['skill_OS'] = 0
data['skill_EM'] = 0

data['diff_1'] = 0
data['diff_2'] = 0
data['diff_3'] = 0
data['diff_4'] = 0
data['diff_5'] = 0

data['no_game'] = 0

for i,r in data.iterrows():
    skill_col = 'skill_'
    diff_col = 'diff_'
    
    if isinstance(r['skill'], str):
        skill_col += r['skill']
        data.at[i, skill_col] = 1
    else:
        data.at[i, 'no_game'] = 1
        
    if not (math.isnan(r['difficulty'])):
        diff_col += str(int(r['difficulty']))
        data.at[i, diff_col] = 1


In [7]:
# drop session 0

data = data[~(data['session_num'] == 0)]  

In [8]:
# normalize between 0 and 1

other_cols = ['games_total', 'games_session', 'mistakes_total', 'mistakes_session', 'mistakes_game', 'ts_robot_talked', 'ts_game_start', 'ts_attempt']

for i in data.columns:
    if 'op_' in i or 'of_' in i or i in other_cols:
        min_val = data[i].min()
        max_val = data[i].max()
        range_val = (float) (max_val - min_val)
        
        data[i] = data[i].fillna(value=min_val)
        data[i] = (data[i] - min_val) / range_val

In [13]:
data = data.round(10)

In [14]:
path_data = '../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Help-Seeking/Data/p8_data_processed2.csv'

data.to_csv(path_data, index=False)